In [ ]:
#@markdown ## **Setup**
import ipywidgets as widgets
from ipywidgets import GridspecLayout
from ipywidgets import AppLayout, Button, Layout, jslink, IntText, IntSlider
import requests
def import_text(text):
  url = 'https://raw.githubusercontent.com/byuccl/digital_design_colab2/master/Exercises/flip_flops/files/%s' %text
  resp = requests.get(url)
  with open(text, 'wb') as f:
    f.write(resp.content)

import_text("tt_ff_key.py")
import_text("Templates/", "tt_template.py")
from tt_ff_key import *
from tt_template import *
create_tt_dictionary(get_tt_data())

## **Sequential Logic**

#### **Clock**

Up until this point we have only been working with combinational logic which doesn't require a clock. The signals changed imedietly and there was no control over when this could happen. This works well when we are wanting a quick responce and dont care about timing. However, when working with a larger system we often need to communicate with other components and need a way to keep everything synchronized. We also dont have a way of storing values in "memory". For these 2 reasons we need to introduce a clock.

The clock is simply a signal that rises to a high state (1) for a period and then drops back to a low state (0) for a period. By oscilating back and forth we can create a "clock" that is ticking back and forth. We can control our system by only allowing signals to change on the rising edge of the clock. This means signals can only change when the clock is going from 0 to 1. A falling edge is when the clock is going from 1 to 0.

You may be thinking that this will make our system way to slow. You are correct that it slows things down. However, modern computers can drive the clocks at extremly fast rates. You have probably seen a computer say that it has a 2.1 GHZ clock speed. This means that the clock rises to 1 and falls back to 0 aproximitly 2,100,000,000 times every second. We will now focus on how we can use this clock to store a value.
<br>



## **SR Latch**

What if we want to store a value?
What if we want to temporarily store a `1` somewhere?
This is where flip flops come in.
It is made up of 2 NOR gates with the output of each one fed back as an input to the other one.
This allows the output `Q` to represent a singe bit that is being stored.

<img src="https://raw.githubusercontent.com/byuccl/digital_design_colab2/master/Exercises/flip_flops/media/sr_latch.png"
width="400" height="250" style="display: block; margin: 0 auto " />

This is called an SR latch.
`S` is the input to "set" `Q`.
If `S` is high for a moment then the `Q` will be set to 1 and stay there.
`R` is the input to "reset" `Q`.
If `R` is high for a moment then `Q` will be reset to 0 and stay there.
The value of `Q` stores a bit.
There is also always a `Q'` value which is the opposite of `Q`, but we don't need to use it for anything.
You can see below what happens when R and S are both low and then S goes high.

<img src="https://raw.githubusercontent.com/byuccl/digital_design_colab2/master/Exercises/flip_flops/media/latch_progression.png"
width="600" height="150" style="display: block; margin: 0 auto " />





This is a good design, but it would be nice to have more control over when our stored value can change.
To do this we can feed the `S` and `R` signals through 2 AND gates along with a `GATE` signal.
In the diagram below the `GATE` signal is called `clk`.

<img src="https://raw.githubusercontent.com/byuccl/digital_design_colab2/master/Exercises/flip_flops/media/gates_sr_latch.png"
width="400" height="200" style="display: block; margin: 0 auto " />

This is called a gated SR latch.
We can set the `clk` signal high if we ant to set or reset `Q`, and then lower it again if we want `Q` to maintain its value no matter what.
This is a better design, but there is still some weird behavior.
Consider what would happen if `R` and `S` are both high at the same time?

### **Truth table**

In [ ]:
#@markdown The truth table for an SR latch is a little different that previous truth tables becuase we use the current output (Q) as an input to determine what the next output will be (Q+).
#@markdown Fill out the truth table below. Assume that the latch is not gated, or that the gate signal is always high. It will help if you draw an SR latch and label `S`, `R`, and `Q` with 1s or 0s depending on the row of the table.
#@markdown Then start with one NOR gate and so if that ouput would change, and then follow the new output to the other NOR gate. If the new output will be undefied, enter 2 in the truth table.
print_tt_ff_grid(1)

## **Flip Flop**

Now let's make one more change to fix the udefinded behavior when S and R are both high. We can eliminate this problem by turning S and R into one signal, D. Then D will go directly into one of the AND gates, and will go through an inverter before connecting to the other AND gate. This is called a D latch.By doing this we make it so that S and R can never both be 1 at the same time. We still have to wait for the clock edge in order to update our stored q signal.

<img src="https://raw.githubusercontent.com/byuccl/digital_design_colab2/master/Exercises/flip_flops/media/gated_d_latch.png"
width="400" height="200" style="display: block; margin: 1000 " />  

Notice that when D is high, it is essentially the Set signal, and when D is low, D` is the Reset signal. However, S was on the bottom of the diagram previously but D is on the top of this new one. As a result of this Q switch from the top output to the bottom output. Try tracing the effect that D has on the circuit when it is high or low. Gated D Latches are so common, it is represented with the simpler symbol below.

<img src="https://raw.githubusercontent.com/byuccl/digital_design_colab2/master/Exercises/flip_flops/media/latch.png"
width="200" height="175" style="display: block; margin: 1000 " />   

There is a small problem though, if `D` is changed too quickly, it can cause a race. The signal will rush from `D` and split. The two different paths will race each other. It is unknown which path will win, especially since the `clk` signal will turn quickly after. This means that we don't know what the value of `Q` will be, which isn't good. When the state of the circuit is unknown in this way, it is called metastability.

<img src="https://raw.githubusercontent.com/byuccl/digital_design_colab2/master/Exercises/flip_flops/media/master_slave.png"
width="350" height="175" style="display: block; margin: 1000 " />  

To prevent metastability and to use the rising clock edge we utilize a Master-Slave latch design. Because our clock signal is being inverted before reaching the slave, only one D latch can change at a time. This means when the clock is high the master is stable while the slave is being updated. When the clock is low the slave is stable and the master is being updated. This means that if were trying to read the value of q, it will be updated on every rising edge. Making this a rising edge flip flop. By using many flip flops we can store a bunch of 1's and 0's and begin to build a circuit that is capable of performing digital logic.

Click [Here](https://colab.research.google.com/github/byuccl/digital_design_colab2/blob/master/Exercises/registers/registers.ipynb) to move on to the Registers Lesson.